In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import evalml
import time

In [2]:
df = pd.read_csv('wustl_hdrl_2024.csv')
df.head()

C:\Users\nakul\AppData\Local\Temp\ipykernel_15384\3850691082.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wustl_hdrl_2024.csv')


,Dir,Flgs,SrcAddr,DstAddr,Sport,Dport,SrcBytes,DstBytes,SrcLoad,DstLoad,...,sswap_sin,sswap_sout,sdiskusage_total,sdiskusage_used,sdiskusage_free,sdiskusage_percent,Boot_Time_with_date,DTime,Attack_categories,Label
0,->,e,192.168.3.2,192.168.2.2,36390,4022,1651.0,272.0,10440.0,1612.000,...,0,0,502392610816,51814645760,424982593536,10,2023-12-23 15:38:11,104.316401,normal,0
1,->,e,192.168.3.2,192.168.2.2,49608,4022,1673.0,272.0,10576.0,1611.000,...,0,0,502392610816,51814674432,424982564864,10,2023-12-23 15:38:11,104.207156,normal,0
2,->,e,192.168.3.2,192.168.2.2,34596,4022,1650.0,272.0,10364.0,1602.000,...,0,0,502392610816,51814686720,424982552576,10,2023-12-23 15:38:11,104.177347,normal,0
3,->,e d,78.151.208.250,192.168.2.2,18346,4022,1450.0,174.0,0.0,306.469,...,0,0,502392610858,50459127848,426338111537,15,2023-05-09 15:19:38,25.821966,DDoS,1
4,->,e d,143.109.155.233,192.168.2.2,18346,4022,1452.0,174.0,0.0,305.268,...,0,0,502392610863,50459127854,426338111539,15,2023-05-09 15:19:38,25.945508,DDoS,1


In [3]:
df['Attack_categories'].value_counts()

Attack_categories
normal             132884
DDoS                 9971
MiTM                 1672
Ransomware            528
Buffer_overflow        68
Name: count, dtype: int64

In [4]:
df.shape

(145123, 77)

In [5]:
df=df.sample(frac=1)
df.reset_index(drop=True,inplace=True)

In [6]:
df = df.loc[:, df.nunique() > 1]
df.shape

(145123, 64)

In [7]:
df.head(10)

,Dir,Flgs,SrcAddr,DstAddr,Sport,SrcBytes,DstBytes,SrcLoad,DstLoad,SrcGap,...,sswap_total,sswap_free,sdiskusage_total,sdiskusage_used,sdiskusage_free,sdiskusage_percent,Boot_Time_with_date,DTime,Attack_categories,Label
0,->,e,198.51.100.100,192.168.4.2,43394,1646.0,272.0,10390.0,1609.0,0.0,...,104853504,104853504,30612852736,17282068480,11999371264,59,2023-12-23 15:40:42,24.397109,normal,0
1,->,e,192.168.6.2,192.168.4.2,43084,1936.0,272.0,13455.0,1595.0,0.0,...,2147479552,2147479552,502392610816,15318241280,461478998016,3,2023-12-23 15:40:19,10.282698,normal,0
2,->,e,192.168.3.2,192.168.2.2,48858,1674.0,272.0,10583.0,1611.0,0.0,...,2147479552,2147479552,502392610816,51705778176,425091461120,10,2023-12-23 15:38:11,104.730351,normal,0
3,->,e,192.168.3.2,192.168.2.2,58006,1672.0,272.0,10567.0,1611.0,0.0,...,2147479552,2147479552,502392610816,51832619008,424964620288,10,2023-12-23 15:38:11,104.822220,normal,0
4,->,e,192.168.6.2,192.168.4.2,51512,1953.0,272.0,13595.0,1598.0,0.0,...,2147479552,2147479552,502392610816,15317737472,461479501824,3,2023-12-23 15:40:19,4.633681,normal,0
5,->,e,198.51.100.100,192.168.4.2,35098,1659.0,272.0,10460.0,1607.0,0.0,...,104853504,104853504,30612852736,19010306048,10271133696,64,2023-12-23 15:40:42,26.820329,normal,0
6,->,e,198.51.100.100,192.168.4.2,54134,1657.0,272.0,10388.0,1598.0,0.0,...,104853504,104853504,30612852736,17634091008,11647348736,60,2023-12-23 15:40:42,24.829487,normal,0
7,->,e,198.51.100.100,192.168.4.2,36694,1648.0,272.0,10401.0,1609.0,0.0,...,104853504,104853504,30612852736,19064852480,10216587264,65,2023-12-23 15:40:42,26.690381,normal,0
8,->,e,192.168.6.2,192.168.4.2,33294,1942.0,272.0,13502.0,1595.0,0.0,...,2147479552,2147479552,502392610816,15326191616,461471047680,3,2023-12-23 15:40:19,1.907572,normal,0
9,->,e,192.168.3.2,192.168.2.2,53524,1675.0,272.0,10583.0,1611.0,0.0,...,2147479552,2147479552,502392610816,51736391680,425060847616,10,2023-12-23 15:38:11,105.832140,normal,0


In [8]:
X = df.drop(['Attack_categories','Label'],axis=1)
y = df['Attack_categories']
y.value_counts()

Attack_categories
normal             132884
DDoS                 9971
MiTM                 1672
Ransomware            528
Buffer_overflow        68
Name: count, dtype: int64

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145123 entries, 0 to 145122
Data columns (total 62 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Dir                        145123 non-null  object 
 1   Flgs                       145123 non-null  object 
 2   SrcAddr                    145123 non-null  object 
 3   DstAddr                    145123 non-null  object 
 4   Sport                      145123 non-null  object 
 5   SrcBytes                   145123 non-null  float64
 6   DstBytes                   145123 non-null  float64
 7   SrcLoad                    145123 non-null  float64
 8   DstLoad                    145123 non-null  float64
 9   SrcGap                     145123 non-null  float64
 10  SIntPkt                    145123 non-null  float64
 11  DIntPkt                    145123 non-null  float64
 12  SIntPktAct                 145123 non-null  float64
 13  DIntPktAct                 14

In [10]:
non_numeric_features = X.select_dtypes(exclude=['number']).columns.tolist()
non_numeric_features

['Dir',
 'Flgs',
 'SrcAddr',
 'DstAddr',
 'Sport',
 'SrcMac',
 'DstMac',
 'RTime',
 'Boot_Time_with_date']

In [11]:
X[non_numeric_features]

,Dir,Flgs,SrcAddr,DstAddr,Sport,SrcMac,DstMac,RTime,Boot_Time_with_date
0,->,e,198.51.100.100,192.168.4.2,43394,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2023-12-30 15:29:32.532481,2023-12-23 15:40:42
1,->,e,192.168.6.2,192.168.4.2,43084,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2023-12-27 05:34:02.626715,2023-12-23 15:40:19
2,->,e,192.168.3.2,192.168.2.2,48858,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2023-12-27 17:43:20.818339,2023-12-23 15:38:11
3,->,e,192.168.3.2,192.168.2.2,58006,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2023-12-28 17:13:08.266855,2023-12-23 15:38:11
4,->,e,192.168.6.2,192.168.4.2,51512,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2024-01-01 09:34:02.363888,2023-12-23 15:40:19
...,...,...,...,...,...,...,...,...,...
145118,->,e,192.168.3.2,192.168.2.2,51832,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2024-01-03 05:13:49.163878,2023-12-23 15:38:11
145119,->,e,192.168.6.2,192.168.4.2,58924,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2023-12-27 17:49:43.100079,2023-12-23 15:40:19
145120,->,e,192.168.6.2,192.168.4.2,58918,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2023-12-30 15:08:35.030337,2023-12-23 15:40:19
145121,->,e,198.51.100.100,192.168.4.2,58254,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2024-01-01 11:23:08.375258,2023-12-23 15:40:42


In [12]:
X.drop(['RTime','Boot_Time_with_date','Dir'],axis=1,inplace=True)

In [13]:
le = LabelEncoder()
X['Sport']=X['Sport'].astype('str')
X['Sport'] = le.fit_transform(X['Sport'])
X['SrcMac'] = le.fit_transform(X['SrcMac'])
X['DstMac'] = le.fit_transform(X['DstMac'])
X['DstAddr'] = le.fit_transform(X['DstAddr'])
y = pd.Series(le.fit_transform(y))

In [14]:
X.drop(['Flgs','SrcAddr','IMEI','sMinPktSz'],inplace=True,axis=1)
X.shape

(145123, 55)

In [15]:
y.value_counts()

4    132884
1      9971
2      1672
3       528
0        68
Name: count, dtype: int64

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [17]:
X_train,X_test,y_train,y_test = train_test_split(np.array(X),y,test_size=0.2,stratify = y)

In [19]:
import time
import tracemalloc
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Assuming X_train, X_test, y_train, y_test are already defined

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Perceptron": Perceptron(),
    "Simple MLP": MLPClassifier(max_iter=1000),
    "SVM": SVC(probability=True),
    "KNN": KNeighborsClassifier(),
    "SGDClassifier": SGDClassifier()
}

# Initialize results dictionary
results = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
    "ROC AUC Score": [],
    "MAE": [],
    "Training Time (s)": [],
    "Memory Used (MB)": []
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    # Start tracking memory and time
    tracemalloc.start()
    start_time = time.time()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    try:
        y_pred_proba = model.predict_proba(X_test)
    except Exception as e:
        print(e)
        pass
    
    # Stop tracking memory and time
    training_time = time.time() - start_time
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr') if y_pred_proba is not None else np.nan
    mae = mean_absolute_error(y_test, y_pred)
    
    # Append results
    results["Model"].append(model_name)
    results["Accuracy"].append(accuracy)
    results["Precision"].append(precision)
    results["Recall"].append(recall)
    results["F1-Score"].append(f1)
    results["ROC AUC Score"].append(roc_auc)
    results["MAE"].append(mae)
    results["Training Time (s)"].append(training_time)
    results["Memory Used (MB)"].append(peak / 10**6)  # Convert to MB


Training Logistic Regression...


C:\Users\nakul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training Random Forest...
Training Decision Tree...
Training Perceptron...
'Perceptron' object has no attribute 'predict_proba'
Training Simple MLP...


C:\Users\nakul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training SVM...
Training KNN...
Training SGDClassifier...
This 'SGDClassifier' has no attribute 'predict_proba'


C:\Users\nakul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)

                 Model  Accuracy  Precision    Recall  F1-Score  \
0  Logistic Regression  0.984806   0.976291  0.585343  0.980345   
1        Random Forest  0.998829   0.998840  0.815673  0.998674   
2        Decision Tree  0.997761   0.997699  0.823214  0.997728   
3           Perceptron  0.984978   0.978042  0.591188  0.981085   
4           Simple MLP  0.996555   0.996354  0.788779  0.996315   
5                  SVM  0.987769   0.985404  0.597119  0.983202   
6                  KNN  0.995624   0.995431  0.779813  0.995329   
7        SGDClassifier  0.986563   0.981745  0.564486  0.982712   

   ROC AUC Score       MAE  Training Time (s)  Memory Used (MB)  
0       0.986076  0.030284           6.077534         11.265877  
1       0.977726  0.002791          33.095166         34.650515  
2       0.910073  0.005650           5.402067         29.908944  
3       0.910073  0.029940           1.118919          2.508323  
4       0.995417  0.007476          53.086004         24.687471  


In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier
import time
base_clf = HistGradientBoostingClassifier()
stime=time.time()
base_clf.fit(X_train,y_train)
print('Time is:',time.time()-stime)

Time is: 1.808504581451416
